In [119]:
from bs4 import BeautifulSoup,Comment
import numpy as np
import pandas as pd
import regex as re
import requests
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import json
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import datetime

# Gas Prices

In [120]:
request_headers = {
    'accept':
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/601.3.9'
}

# Set up Splinter
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

with requests.Session() as session:
    url = 'https://gasprices.aaa.com/'
    response = session.get(url, headers=request_headers)

# browser.visit(url)
# # Optional delay for loading the page
# browser.is_element_present_by_css('div.list_text', wait_time=5)

In [189]:
soup = BeautifulSoup(response.content, 'html.parser')
df = pd.DataFrame()

html_data = soup.find(class_="table-mob")
html_data2 = soup.find(class_="average-price")

date = [span.get_text() for span in html_data2.find_all("span")]
data = [td.get_text() for td in html_data.find_all("td")]

CurrentDate = []
GasPrice = []
DieselPrice = []
day = pd.to_datetime(date[0][11:])

CurrentDate.append(day)
GasPrice.append(round(float(data[1][1:]),2))
DieselPrice.append(round(float(data[4][1:]),2))

df['Date'] = CurrentDate
df['Regular'] = GasPrice
df['Diesel'] = DieselPrice

gas = pd.read_csv("./gas-diesel-prices.csv")
gas['Date'] = pd.to_datetime(gas['Date'])
gas = pd.concat([gas, df], ignore_index = True)
gas = gas.drop(columns=['Unnamed: 0'])
gas = gas.drop_duplicates()
gas

,Date,Regular,Diesel
0,2022-02-23,3.54,3.96
1,2022-02-24,3.54,3.96
2,2022-02-25,3.57,3.98
3,2022-02-26,3.60,4.00
4,2022-02-27,3.60,4.00
...,...,...,...
291,2022-12-14,3.21,4.86
292,2022-12-15,3.19,4.83
293,2022-12-16,3.18,4.81
294,2022-12-17,3.16,4.79


In [190]:
gas.to_csv('gas-diesel-prices.csv')

# Oil Prices

In [56]:
request_headers = {
    'accept':
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/601.3.9'
}

# # Set up Splinter
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

with requests.Session() as session:
    url = 'https://www.marketwatch.com/investing/future/cl.1'
    response2 = session.get(url, headers=request_headers)

# browser.visit(url)
# # Optional delay for loading the page
# browser.is_element_present_by_css('div.list_text', wait_time=5)

In [202]:
soup2 = BeautifulSoup(response2.content, 'html.parser')
df2 = pd.DataFrame()

html_data3 = soup2.find('th', attrs={'class':'table__heading'})
html_data4 = soup2.find('td', attrs={'class':'table__cell u-semi'})

html_data3 = html_data3.text.replace('\n    {\n        ','{')
html_data3 = html_data3.replace('\n    }\n    ','}')
html_data4 = html_data4.text.replace('\n    {\n        ','{')
html_data4 = html_data4.replace('\n    }\n    ','}')
pre_date = pd.to_datetime(html_data3[17:])
pre_price = html_data4[1:]

date = []
crude = []
date.append(pre_date)
crude.append(round(float(pre_price),2))
df2["Date"] = date
df2["Crude Closing"] = crude

oil_df = pd.read_csv("./oil-prices.csv")
oil_df['Date'] = pd.to_datetime(oil_df['Date'])
oil_df = pd.concat([oil_df, df2], ignore_index = True)
oil_df = oil_df.drop(columns=['Unnamed: 0'])

oil_df = oil_df.drop_duplicates()
oil_df

,Date,Crude Closing
0,2022-01-03,76.08
1,2022-01-04,76.99
2,2022-01-05,77.85
3,2022-01-06,79.46
4,2022-01-07,78.90
...,...,...
237,2022-12-12,73.17
238,2022-12-13,75.39
239,2022-12-14,77.28
240,2022-12-15,76.11


In [50]:
oil_df.to_csv('oil-prices.csv')

# One DataFrame

In [130]:
oil_gas_df = pd.merge(gas, oil_df, on=["Date", "Date"])

In [131]:
oil_gas_df

,Date,Regular,Diesel,Crude Closing
0,2022-02-23,3.54,3.96,92.10
1,2022-02-24,3.54,3.96,92.81
2,2022-02-25,3.57,3.98,91.59
3,2022-02-28,3.61,4.01,95.72
4,2022-03-01,3.62,4.02,103.41
...,...,...,...,...
201,2022-12-12,3.26,4.92,73.17
202,2022-12-13,3.25,4.89,75.39
203,2022-12-14,3.21,4.86,77.28
204,2022-12-15,3.19,4.83,76.11


In [205]:
oil_gas_df.index = oil_gas_df.index.map(str)
oil_gas_dict = oil_gas_df.to_dict()